# Анализ NPS у клиентов 

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 


Для того чтобы определить уровень лояльности, был проведен опрос в котором клиентам задавали классический вопрос:  
`«Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым»`.  

`Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков»`

Данные с результатами загружены в БД SQLite.  

Описание данных
Таблица `user`
Содержит основную информацию о клиентах.
- `user_id` - идентификатор клиента, первичный ключ таблицы
- `lt_day` - количество дней «жизни» клиента
- `age` - возраст клиента в годах
- `gender_segment` - пол клиента (1 – женщина, 0 – мужчина)
- `os_name` - тип операционной системы
- `cpe_type_name` - тип устройства
- `location_id` - идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- `age_gr_id` - идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- `tr_gr_id` - идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице `traffic_segment`
- `lt_gr_id` - идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице `lifetime_segment`
- `nps_score` - оценка клиента в NPS-опросе (от 1 до 10)  

Таблица `location`
Справочник территорий, в которых телеком-компания оказывает услуги.
- `location_id` - идентификатор записи, первичный ключ
- `country`	- страна
- `city` - город  

Таблица `age_segment`
Данные о возрастных сегментах клиентов.  
- `age_gr_id` - идентификатор сегмента, первичный ключ
- `bucket_min` - минимальная граница сегмента
- `bucket_max` - максимальная граница сегмента
- `title` - название сегмента  

Таблица `traffic_segment`
Данные о выделяемых сегментах по объёму потребляемого трафика.
- `tr_gr_id` - идентификатор сегмента, первичный ключ
- `bucket_min` - минимальная граница сегмента
- `bucket_max` - максимальная граница сегмента
- `title` - название сегмента  

Таблица `lifetime_segment`
Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.
- `lt_gr_id` - идентификатор сегмента, первичный ключ
- `bucket_min` - минимальная граница сегмента
- `bucket_max` - максимальная граница сегмента
- `title` - название сегмента  

Мы будем объединять все данные в одну таблицу, проводить анализ и готовить презентацию с помощью `Tableau`.     

Ссылку на готовую презентацию отправим клиенту.

## 1 Подключение библиотек и подключение к БД

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## 2 Смотрим на таблицы перед их объединением

Псмотрим на данные перед их объединением

### 2.1 Таблица user

In [3]:
query = """
select * from user
"""

In [4]:
df_user = pd.read_sql(query, engine)
df_user.head()

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10


In [5]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 42.2+ MB


In [6]:
df_user.describe()

,lt_day,age,gender_segment,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
count,502493.000000,501939.000000,501192.000000,502493.000000,502493.000000,502493.00000,502493.000000,502493.000000
mean,1868.841439,39.621946,0.543588,31.237321,4.011045,8.51829,6.890339,7.508562
std,1683.701762,11.188249,0.498097,15.175477,1.159203,4.64590,1.525189,3.020378
min,-21.000000,10.000000,0.000000,1.000000,1.000000,1.00000,1.000000,1.000000
25%,533.000000,31.000000,0.000000,20.000000,3.000000,5.00000,6.000000,5.000000
50%,1239.000000,38.000000,1.000000,30.000000,4.000000,7.00000,8.000000,9.000000
75%,3064.000000,47.000000,1.000000,41.000000,5.000000,10.00000,8.000000,10.000000
max,9162.000000,89.000000,1.000000,62.000000,8.000000,25.00000,8.000000,10.000000


### 2.2 Таблица location

In [7]:
query = """
select * from location
"""

In [8]:
df_loc = pd.read_sql(query, engine)
df_loc.head()

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия


In [9]:
df_loc['city'].value_counts()

Череповец    1
Саратов      1
Иваново      1
Курск        1
Сургут       1
            ..
Грозный      1
Уфа          1
Волжский     1
Кемерово     1
УланУдэ      1
Name: city, Length: 62, dtype: int64

### 2.3 Таблица age_segment

In [10]:
query = """
select * from age_segment
"""

In [11]:
df_age = pd.read_sql(query, engine)
df_age

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


### 2.4 Таблица traffic_segment

In [12]:
query = """
select * from traffic_segment
"""

In [13]:
df_trf = pd.read_sql(query, engine)
df_trf

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


### 2.5 Таблица lifetime_segment

In [14]:
query = """
select * from lifetime_segment
"""

In [15]:
df_life = pd.read_sql(query, engine)
df_life

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


### 3 Объединяем все данные

In [16]:
query = """
select user_id, lt_day, 
        case 
            when u.lt_day <= 365 then 'новый'
            when u.lt_day > 365 then 'старый'
            else 'не известно'
            end as is_new, 
        age, 
        case
            when gender_segment = 1 then 'женщина'
            when gender_segment = 0 then 'мужчина'
            else 'не известно'
        end as gender_segment, 
        os_name, cpe_type_name, 
        country, city, 
        a.title as age_segment, 
        t.title as traffic_segment, 
        lt.title as lifetime_segment, 
        nps_score,
        case
            when nps_score in (9,10) then 'promoters'
            when nps_score in (7,8,9) then 'passives'
            when nps_score in (0,1,2,3,4,5,6) then 'detractors'
            end as nps_group
 from user as u 
 left join location as l on u.location_id = l.location_id
 left join age_segment as a on u.age_gr_id = a.age_gr_id
 left join traffic_segment as t on u.tr_gr_id = t.tr_gr_id
 left join lifetime_segment as lt on u.lt_gr_id = lt.lt_gr_id
"""

Смотрим на получившиеся данные

In [17]:
df = pd.read_sql(query, engine)
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,старый,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,старый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,старый,54.0,женщина,ANDROID,SMARTPHONE,Россия,Москва,05 45-54,04 1-5,08 36+,5,detractors
502489,ZZZLWY,129,новый,31.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,03 25-34,04 1-5,04 4-6,8,passives
502490,ZZZQ5F,522,старый,36.0,мужчина,ANDROID,SMARTPHONE,Россия,Сургут,04 35-44,09 25-30,06 13-24,10,promoters
502491,ZZZQ8E,2936,старый,37.0,женщина,ANDROID,SMARTPHONE,Россия,УланУдэ,04 35-44,17 65-70,08 36+,9,promoters


Проверяем типы данных и наличие пропусков в данных

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [19]:
df.isna().mean()

user_id             0.000000
lt_day              0.000000
is_new              0.000000
age                 0.001103
gender_segment      0.000000
os_name             0.000000
cpe_type_name       0.000000
country             0.000000
city                0.000000
age_segment         0.000000
traffic_segment     0.000000
lifetime_segment    0.000000
nps_score           0.000000
nps_group           0.000000
dtype: float64

Итак, в данных мы видим пропуски в колонках 
- `age` (0.11%)
- `gender_segment` (0.26%). 

Заменить пропуски значениями мы не можем, но их наличие не повлияет на результат нашего исследования, так как их количество незначительно

Проверим к какой группе отнесены пропуски в колонке `age`

In [20]:
df.query('age.isna()')['age_segment'].value_counts()

08 n/a    554
Name: age_segment, dtype: int64

Название группы для отсутствующих значений подобрано корректно

В колонках `age_segment`, `age_segment`, `lifetime_segment` значения начинаются с порядкового номера группы. Его оставим для того чтобы группы были отсортированы по порядку на графиках в презентации

## Выгрузка данных для анализа в Tableau

In [21]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Презентация с результатами анализа NPS клиентов

Выводы полученные в ходе анализа данных
1. Общий это показатель уровня лояльности клиентов NPS для клиентов оператора составляет 21,9% и рассчитывается на основе ответов на вопросы об их готовности рекомендовать компанию другим.

2. Исходя из опроса, клиенты, относящиеся к группе "промоутеры" , в основном - это Москвичи в возрасте от 35 до 45 лет, пользующиеся операционной системой Android и использующие от 1 до 5 гигабайт трафика.

3. Распределение NPS характеризуется некоторыми особенностями: лояльность клиентов возрастает с возрастом, с достижением максимума среди людей старшего поколения. Значения NPS выше среди женщин, чем среди мужчин во всех возрастных категориях. Также наблюдается снижение NPS при увеличении стажа абонента и более высокие значения в небольших городах. Однако, NPS снижается при росте объема скачанного трафика и при низком объеме трафика. Значения NPS примерно одинаковы для всех операционных систем, за исключением клиентов с iOS, где значение значительно ниже.

4. Выводы и наблюдения по распределению клиентов оператора в опросе: в опросе приняло участие 500000 человек из 62 городов, с наибольшим количеством участников в возрастной категории от 25 до 44 лет. Количество опрошенных женщин превышает количество мужчин во всех возрастных категориях. Более 90% участников опроса являются абонентами с стажем свыше 1 года. Распределение опрошенных по городам соответствует распределению населения. Операционная система Android является самой популярной среди участников опроса, за ней идет iOS.

Ссылка на презентацию

https://public.tableau.com/shared/4D8PFMCW9?:display_count=n&:origin=viz_share_link